# Finding causal genotype-phenotype relations with ddlite: extraction

## Introduction
In this example **ddlite** app, we'll build a system to indentify causal relationships between genotypes and phenotypes from raw journal articles. For an end-to-end example, see **GeneTaggerExample_Extraction.ipynb** and **GeneTaggerExample_Learning.ipynb**.

In [1]:
%load_ext autoreload
%autoreload 2

import cPickle, os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from ddlite import *

## Processing the input data
We already downloaded the raw HTML for ??? relevant article pages from PubMed. These can be found in the `data` folder. We can use ddlite's `DocParser` to read in the article text. It uses CoreNLP via ddlite's `SentenceParser` to parse each sentence. This can take a little while, so if the example has already been run, we'll reload it.

In [2]:
dp = DocParser('gene_phen_relation_example/data/', TextParser())

pkl_f = 'gene_phen_relation_example/gene_phen_saved_sents_v1.pkl'
try:
    with open(pkl_f, 'rb') as f:
        sents = cPickle.load(f)
except:
    %time sents = dp.parseDocSentences()
    with open(pkl_f, 'w+') as f:
        cPickle.dump(sents, f)

print sents[0]

Sentence(words=[u'Common', u'Inherited', u'Variation', u'in', u'Mitochondrial', u'Genes', u'is', u'not', u'Enriched', u'for', u'Associations', u'with', u'Type', u'2', u'Diabetes', u'or', u'Related', u'Glycemic', u'Traits'], lemmas=[u'common', u'inherit', u'variation', u'in', u'mitochondrial', u'gene', u'be', u'not', u'enrich', u'for', u'association', u'with', u'type', u'2', u'Diabetes', u'or', u'related', u'glycemic', u'trait'], poses=[u'JJ', u'VBN', u'NN', u'IN', u'JJ', u'NNS', u'VBZ', u'RB', u'VBN', u'IN', u'NNS', u'IN', u'NN', u'CD', u'NNP', u'CC', u'JJ', u'JJ', u'NNS'], dep_parents=[3, 3, 9, 6, 6, 3, 9, 9, 0, 11, 9, 13, 11, 15, 13, 15, 18, 15, 15], dep_labels=[u'amod', u'amod', u'nsubjpass', u'case', u'amod', u'nmod', u'auxpass', u'neg', u'ROOT', u'case', u'nmod', u'case', u'nmod', u'nummod', u'dep', u'cc', u'amod', u'conj', u'dep'], sent_id=0, doc_id=0, text='Common Inherited Variation in Mitochondrial Genes is not Enriched for Associations with Type 2 Diabetes or Related Glycemic

## Extracting relation candidates with matchers
Extracting candidates for relations in ddlite is done with `Matcher` objects. Here, we'll use two `DictionaryMatcher`s. We have access to pretty comprehensive gene and phenotype dictionaries. Let's load them in and create the `DictionaryMatcher`s.

In [3]:
# Schema is: ENSEMBL_ID | NAME | TYPE (refseq, canonical, non-canonical)
genes = [line.rstrip().split('\t')[1] for line in open('gene_phen_relation_example/dicts/ensembl_genes.tsv')]
genes = filter(lambda g : len(g) > 3, genes)

# Schema is: HPO_ID | NAME | TYPE (exact, lemma)
phenos = [line.rstrip().split('\t')[1] for line in open('gene_phen_relation_example/dicts/pheno_terms.tsv')]

GM = DictionaryMatch('G', genes, ignore_case=False)
PM = DictionaryMatch('P', phenos)

If we wanted to define more `Matcher` for, say, genes, we could use multiple `Matcher` objects with a `MultiMatcher`. For now, we'll just use the single `DictionaryMatcher` for both classes. We'll use this to extract our candidate relations from the sentences into an `Relations` object. Using just the matchers will likely provide high recall but poor precision. This is because not all genotype-phenotype mention pairs in the same sentence represent a causal pairing. The `Relations` object we create can be used in a `CandidateModel`. This allows us to learn a model to predict whether each candidate pair represents a causal pair or not.

In [7]:
R = Relations(sents, GM, PM)

We can visualize contexts for our extractions too. This may help in writing labeling functions in a learning task.

In [12]:
R[2].render()

<!--Provide the canvas id (twice) and the words via python string formatting here--!>
 
 
 Two neuropathies , Charcot-Marie-Tooth type 2A and autosomal dominant optic atrophy , are caused by mutations in essential fusion components , namely , mitofusin 2 or OPA1 .

Finally, we can pickle the extracted candidates from our `Relations` object for later use.

In [13]:
R.dump_candidates('gene_phen_relation_example/gene_phen_saved_relations_v1.pkl')